In [1]:
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
import spacy

In [2]:
# converting strings into unicode
df = pd.read_csv('Quora.csv')

df['question1']= df['question1'].apply(lambda x : str(x));
df['question2'] = df['question2'].apply(lambda x : str(x));

In [3]:
# Applying Tf- iDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

questions = list(df['question1'] + df['question2'])
tf = TfidfVectorizer(lowercase=False,)
tf.fit_transform(questions)

word2tfidf = dict(zip(tf.get_feature_names(),tf.idf_)) #DOUBT

In [4]:
import en_core_web_sm

In [5]:
nlp = en_core_web_sm.load()

In [6]:
#contains final for questions 
vect1 = []
#DOUBT
for question in tqdm(list(df['question1'])):
    doc1 = nlp(question)
    
    ques_mean1 = np.zeros([len(doc1),len(doc1[0].vector)]) 
    for word in doc1:
        w1 = word.vector
        
        try:
            idf = word2tfidf[str(word)]
        except:
            idf = 0
            
        ques_mean1 += w1*idf
    ques_mean1 = ques_mean1.mean(axis = 0)
    vect1.append(ques_mean1)
df['q1_feat_tf'] = list(vect1)    

100%|████████████████████████████████████████████████████████████████████████| 404290/404290 [1:38:16<00:00, 68.57it/s]


In [7]:
#contains final for questions 
vect2 = []

for question in tqdm(list(df['question2'])):
    doc2 = nlp(question)
    
    ques_mean2 = np.zeros([len(doc2),len(doc2[0].vector)]) 
    for word in doc2:
        w2 = word.vector
        
        try:
            idf = word2tfidf[str(word)]
        except:
            idf = 0
            
        ques_mean2 += w2*idf
    ques_mean2 = ques_mean2.mean(axis = 0)
    vect2.append(ques_mean2)
df['q2_feat_tf'] = list(vect2)

100%|████████████████████████████████████████████████████████████████████████| 404290/404290 [2:43:55<00:00, 41.10it/s]


In [8]:
df.to_pickle('Files/spacy.csv')

In [11]:
if os.path.isfile('Files/spacy.csv'):#where is spacy file saved?
    df_spacy = pd.read_pickle('Files/spacy.csv')
else :
    print("Run from the beginning of the file")

In [9]:
if os.path.isfile('Files/nlp_features_train.csv'):
    df_nlp = pd.read_csv('Files/nlp_features_train.csv',encoding='latin-1') #will it work if directly download files? or have to run
else :
    print("The nlp data set features are not found")
    
if os.path.isfile('Files/df_features_train.csv'):
    df_basic = pd.read_csv('Files/df_features_train.csv') #WHICH FILES??
else:
    print("Df_features_train.csv not found")
    print('_________________EXIT___________________')
    

In [15]:
df1 = df_nlp.drop(['qid1','qid2','question1','question2'],axis=1)#DOUBT
df2 = df_basic.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df_spacy.drop(['qid1','qid2','question1','question2','is_duplicate'],axis = 1)

# Hence getting Each 300+ features as individual property
df_q1_feat = pd.DataFrame(df3.q1_feat_tf.values.tolist(),index=df3.index)
df_q2_feat = pd.DataFrame(df3.q2_feat_tf.values.tolist(),index=df3.index)

In [16]:
df_q1_feat.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,-20.420960,25.809565,-120.875930,-120.089934,47.743297,82.881647,13.692647,-2.328330,-56.527910,-141.456537,...,-81.454172,95.711299,23.484497,71.713856,-15.987871,-5.651370,132.943438,-133.359732,12.202677,-20.588167
1,-103.531553,74.353032,-131.535842,-121.343942,-28.826116,71.306030,21.399175,16.965575,-3.343543,-34.434352,...,-146.823464,53.217788,109.906141,65.295377,-127.780162,5.002112,59.177660,-120.263983,-71.359894,20.966675
2,-93.197456,-97.906234,-77.455222,-125.958146,57.201867,51.768824,1.641915,39.578487,23.169117,-57.100650,...,-105.966230,118.966684,38.612364,85.922869,-52.499269,32.288933,-5.439048,-115.920739,-5.230780,-3.932547
3,8.359163,-62.501191,-98.837463,-30.157859,26.721602,130.996214,7.831641,38.369273,-25.347790,-41.314360,...,59.643758,24.050001,-67.307009,21.425892,48.925113,-84.020696,28.958123,-61.928086,-79.103757,4.287268
4,-72.378577,1.436893,-180.962019,-214.450679,133.476621,13.439506,-38.458427,100.236844,79.266830,-198.363985,...,-129.560201,164.730044,34.722580,129.237790,-31.972835,141.800667,105.816798,-165.177456,-62.618496,-153.124696


In [ ]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    df_q1_feat['id']=df1['id']
    df_q2_feat['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df_q1_feat.merge(df_q2_feat, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('Files/final_features.csv')